# Pipelines in Sklearn  

In this notebook I am demonstrating the use of **pipelines** in scikit-learn.  

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
test = pd.read_csv("../input/interview/test.csv")
train = pd.read_csv("../input/interview/train.csv")
train.head()

In [ ]:
# Easy dirty way of creating features
def features(df):
    df['dv'] = df['n_vehicles'] / df['n_drivers']
    df['di'] = df['income'] / df['n_drivers']
    df['ci'] = df['income'] / df['cost_of_ad']
features(train)
features(test)

# Categorizing features 
cat_cols = ['gender', 'in_initial_launch_location', 'device_type']
response = ['outcome']
num_cols = [x for x in train.columns if x not in cat_cols + response]

y = train['outcome']
X = train.drop('outcome', axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=10, stratify=y)

In [ ]:
# Binary categoricals
# Imputing missing gender values
# in_initial_launch_location encoded as (0, 1)
# gender encoded as {0: female, 1: missing value, 2: male}
cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='G')),
    ('ordinal', OrdinalEncoder())
])

# One-hot encoding device feature as it has multiple values
device_transformer = Pipeline([
    ('onehot', OneHotEncoder(sparse=False))
])

# Numbers are scaled using Z-score
num_transformer = Pipeline([
    ('scaler', StandardScaler())
])

# Combining the transformers of all feature groups
preprocessor = ColumnTransformer([
    ('cat', cat_transformer, ['gender', 'in_initial_launch_location']),
    ('device', device_transformer, ['device_type']),
    ('num', num_transformer, num_cols)
])

# Preprocessing + Model
hgb = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', HistGradientBoostingClassifier(random_state=7))
])

# Parameter distribution used for randomized search hyperparameter optimization
param_dist = {
    'classifier__learning_rate': np.logspace(-3, 100, 10),
    'classifier__max_iter': np.logspace(1, 4, 20, dtype='int'),
    'classifier__max_depth': [2, 10, 100, 500, 1000],
    'classifier__max_leaf_nodes': np.logspace(1, 2, 7, dtype='int'),
    'classifier__min_samples_leaf': np.logspace(0, 2, 20, dtype='int'),
    'classifier__l2_regularization': np.linspace(0, 1, 10)
}

# Fit and predict
CV = RandomizedSearchCV(hgb, param_dist, random_state=17)
CV.fit(X_train, y_train)
pred = CV.predict(test)

### AUC Estimation with Validation Set  

In [ ]:
y_val_prob = CV.predict_proba(X_val)

In [ ]:
roc_auc_score(y_val, y_val_prob[:, 1])